In [ ]:
import flopy as fp
import flopy.utils as fu
import pandas as pd
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Navigate some constants

In [ ]:
parcels = np.loadtxt('swb_files/parcels.asc', skiprows=6)
# which parcels have irrigation?
irrigation = np.loadtxt('swb_files/irrigation.asc', skiprows=6)
irrigated_parcels = np.unique(parcels[irrigation==1])

# Update the file of land use from a table

In [ ]:
# read in a table of new crops by parcel
new_crops = pd.read_csv('crop_parcel_input.csv', index_col=0)
# also read in the starting land use
landuse = np.loadtxt('swb_files/land_use.asc', skiprows = 6)
# read in the header for writing out rasters as well
raster_header = open('swb_files/land_use.asc', 'r').readlines()[:6]

In [ ]:
new_crops

In [ ]:
# update the landuse array with the new crop types
for cp in new_crops.index:
    landuse[parcels==cp] = new_crops.loc[cp].CropType
# and save it out
np.savetxt('swb_files/land_use.asc', landuse, header=''.join(raster_header).rstrip(), comments='', fmt = '%d')

## first run SWB

In [ ]:
os.system('swb2 freyberg_delta.ctl')

## Next read in the recharge and pumping for a given year

In [ ]:
runyear = 2015

In [ ]:
rchfile = 'net_infiltration__2010-01-01_to_2017-12-31__40_by_20.nc'
pumping_file = 'irrigation__2010-01-01_to_2017-12-31__40_by_20.nc'

In [ ]:
rch = xr.open_dataset(rchfile)
pump = xr.open_dataset(pumping_file)


In [ ]:
pump_data = pump['irrigation'].loc['1/1/{0:d}'.format(runyear):
                                      '12/31/{0:d}'.format(runyear)].sum(axis=0).values
rch_data = rch['net_infiltration'].loc['1/1/{0:d}'.format(runyear):
                                      '12/31/{0:d}'.format(runyear)].sum(axis=0).values
pump.close()
rch.close()

## Unit conversions - the bane of the world!
convert `rch` from inches to meters  
also convert `pump` from inches/year/model cell to cubic meters per day  
$rch_{\frac{m}{day}} = rch_{\frac{in}{year}} \times 0.0254 \times \frac{1 year}{365.25 days}$  

$pump_{\frac{m^3}{d}} = pump_{inches/year/cell} \times 0.0254 \frac{m}{in} \times 
            250^2m^2 \times \frac{1 year}{365.25 days}$

In [ ]:
rch_data *= 0.0254 / 356.25

In [ ]:
pump_data = pump_data * 0.0254 * 250 * 250 /365.25

### read in the parcels and find pumping per parcel (also read in row and col for pumping)

In [ ]:
rc = pd.read_csv('parcel_centroids.dat')
parcels = np.loadtxt('swb_files/parcels.dat').astype(int)

In [ ]:
total_pumping = pd.DataFrame({'parcel':[int(i) for i in np.unique(parcels)],
                         'flux':[-np.sum(pump_data[parcels==i]) for i in np.unique(parcels)]})

In [ ]:
total_pumping

## read in the model

In [ ]:
sim = fp.mf6.MFSimulation.load('mfsim.nam', sim_ws='freyberg_demo')
m = sim.get_model(model_name='freyberg')

In [ ]:
m.remove_package('rch')
m.remove_package('wel')

### update the pumping in wells

In [ ]:
total_pumping = total_pumping.merge(rc, left_on='parcel',right_on='parcel')

In [ ]:
total_pumping=total_pumping.loc[total_pumping.flux<0]
total_pumping.r=[int(np.round(i)) for i in total_pumping.r]
total_pumping.c=[int(np.round(i)) for i in total_pumping.c]
total_pumping['cellid'] = [(0,i,j) for i,j in zip(total_pumping.r,total_pumping.c)]
total_pumping

In [ ]:
well_spd = total_pumping[['cellid','flux']].to_records(index=False)
wel = fp.mf6.ModflowGwfwel(m, print_input=True, stress_period_data=
                           {i:well_spd for i in range(sim.tdis.nper.get_data())})

### update the recharge array

In [ ]:
rch = fp.mf6.ModflowGwfrcha(m, recharge={i:rch_data for i in range(sim.tdis.nper.get_data())})

# Now run and process MODFLOW6

In [ ]:
sim.exe_name='mf6'
sim.write_simulation()

In [ ]:
sim.run_simulation()

In [ ]:
headfile = 'freyberg_demo/freyberg.hds'
hds = fu.binaryfile.HeadFile(headfile)
t = hds.get_times()
t

In [ ]:
h=hds.get_data(totim=t[1])
h.shape

In [ ]:
h[h>1000] = np.nan
h[h<-1000] = np.nan


In [ ]:
plt.figure(figsize=(6,10))
plt.imshow(55-np.squeeze(h))
plt.colorbar()
plt.title('Depth to Water')
plt.savefig('DTW.png')

In [ ]:
plt.figure(figsize=(6,10))
plt.imshow(np.squeeze(h))
plt.colorbar()
plt.title('Water Elevation')
plt.savefig('WaterElevation.png')

In [ ]:
from pumping_cost import pumping_cost

In [ ]:
def DTW_and_cost_by_parcel(parcels, h, land_surface, irrigated_parcels, state='ms'):
    DTW = land_surface - np.squeeze(h)
    parcels_all = [int(i) for i in irrigated_parcels]
    DTW_all = [np.nanmax(DTW[parcels==p]) for p in irrigated_parcels]
    cost = [pumping_cost(i,2015,'ms') for i in DTW_all]
    return pd.DataFrame({'parcel':parcels_all, 'DTW':DTW_all,'pumping_cost':cost})

In [ ]:
DTW_and_cost_by_parcel(parcels,h,55, irrigated_parcels)

In [ ]:
pumping_cost(150, 2015,'ms') # args are lift in feet, year, and state (ms or ar). Returns $/acre-foot